In [1]:
import numpy as np
import threading
import concurrent.futures

In [2]:
data = np.loadtxt("puzzle.txt", dtype=np.int16)
emptycells=[]

In [3]:
#finding empty cells and append them to the array
for row in range(9): 
    for col in range(9): 
        if(data[row][col]==0): 
            emptycells.append([row,col])

#print grid
def grid(arr): 
    for i in range(9): 
        for j in range(9): 
            print(arr[i][j], end = ' ')
        print() 
        
def check_possibility_in_row(arr,row,num):
    for i in range(9):
        if(arr[row][i] == num):
            return False
    return True

def check_possibility_in_col(arr,col,num):
    for i in range(9):
        if(arr[i][col] == num):
            return False
    return True
        
def check_possibility_in_subsquare(arr,row,col,num):
    if((row+1)%3== 0):
        row = row - 2
    else:
        row = row + 1 - (row+1)%3
        
    if((col+1)%3== 0):
        col = col - 2  
    else:
        col = col + 1 - (col+1)%3  
        
    for i in range(3): 
        for j in range(3): 
            if(arr[i+row][j+col] == num):
                return False
    return True

In [4]:
i=0
start_value = 1
filled = 0

if __name__ == "__main__":
    while(i<len(emptycells)):
        row = emptycells[i][0]
        col = emptycells[i][1] 
        for num in range(start_value,10):

            with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
                possibility_col = executor.submit(check_possibility_in_col, data,col,num).result()
                possibility_row = executor.submit(check_possibility_in_row, data,row,num).result()
                possibility_subsquare = executor.submit(check_possibility_in_subsquare, data,row,col,num).result()

            if(possibility_row and possibility_col and possibility_subsquare):
                data[row][col] = num
                start_value = 1
                filled = 1
                break

        if(filled == 0):
            i = i-1
            row = emptycells[i][0]
            col = emptycells[i][1]        
            start_value = data[row][col] + 1
            data[row][col] = 0
        else:
            i = i+1
            filled = 0

In [5]:
grid(data)

3 1 6 5 7 8 4 9 2 
5 2 9 1 3 4 7 6 8 
4 8 7 6 2 9 5 3 1 
2 6 3 4 1 5 9 8 7 
9 7 4 8 6 3 1 2 5 
8 5 1 7 9 2 6 4 3 
1 3 8 9 4 7 2 5 6 
6 9 2 3 5 1 8 7 4 
7 4 5 2 8 6 3 1 9 
